# Laboratory 2.6: LOO + k-Fold Cross Validation

In this practice we will implement one of the main techniques to prevent overfitting when training a model: **cross-validation**.

In addition, we will be using the following libraries:
- Data management:
    - [numpy](https://numpy.org/)
- Modelling and scoring:
    - [scikit-learn](https://scikit-learn.org)
- Plotting:
    - [matplotlib](https://matplotlib.org/)
    
### **All the things you need to do are marked by a "TODO" comment nearby. Make sure you *read carefully everything before working* and solve each point before submitting your solution.**

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os
import sys
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path
sys.path.insert(0, project_root)

In the following cell import the training (`training.dat`) and test (`test.dat`) dataset. We recommend you to use the `np.loadtxt()` function.

You will need to create the `X_train`, `y_train`, `X_test` and `y_test` variables. Take into account that each dataset have 10 variables, where the last one is the output variable.

In [3]:
# TODO: Load training.dat and text.dat and create X_train, y_train, X_test and y_test
train = np.loadtxt("../data/training.dat", delimiter=",")
X_train = train[:, :-1]
y_train = train[:, -1:].reshape(-1)
test = np.loadtxt("../data/test.dat", delimiter=",")
X_test = test[:, :-1]
y_test = test[:, -1:].reshape(-1)

With this data you are going to train the optimal `KNeighborsClassifier` model.

### Initial guess

As you have no idea of what the optimal value of `n_neighbors` is, trust your professors and use `n_neighbors = 4` to train your model.

In [4]:
# TODO: Train a KNeighborsClassifier model with n_neighbors=4
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

Now, calculate the accuracy of the model for the training and test sets using the `accuracy_score` function from `scikit-learn`.

In [5]:
# TODO: Calculate accuracy in training and test for KNN with k=4
acc_train = accuracy_score(y_train, model.predict(X_train))
acc_test = accuracy_score(y_test, model.predict(X_test))
print(f'Accuracy, train = {acc_train} test = {acc_test}')

Accuracy, train = 0.84375 test = 0.785


**What is happening with this value of `n_neighbors`?**
> El modelo tiende al overfitting, ya que tiene peor precisión en el test a cambio de buena precisión en el train, lo cual no nos interesa demasiado

### Damage control
It seems that `n_neighbors = 4` overfits for this dataset. Let's try to correct this and use `n_neighbors = 20`

In [6]:
# TODO: Train a KNeighborsClassifier model with n_neighbors=20
model = KNeighborsClassifier(n_neighbors=20)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

Calculate again the accuracy of training and test sets for this model

In [7]:
# TODO: Calculate accuracy in training and test for KNN with k=20
acc_train = accuracy_score(y_train, model.predict(X_train))
acc_test = accuracy_score(y_test, model.predict(X_test))
print(f'Accuracy, train = {acc_train} test = {acc_test}')

Accuracy, train = 0.81 test = 0.795


It seems that the accuracy of the test set has improved, but how can we be sure that `n_neighbors=20` is the optimal value?

### Obtaining optimal value for hyperparameters

We could keep trying with different values for `n_neighbors` until we find the optimal one. However, this strategy is unfeasible for real datasets. So how can we obtain a reasonable optimal value for the hyperparameters of a model?

We can leverage the power of cross-validation:

<center> <img title="5-Fold Cross-Validation " alt="cross-validation" src="https://miro.medium.com/v2/resize:fit:1200/1*AAwIlHM8TpAVe4l2FihNUQ.png"> </center>

Cross-validation give us a notion of the generalization error (i.e., test error) using parts of the training set as a validation set. This validation set is data that the trained model has not seen before, so if the model performs well in this part of the dataset it should generalize well in unseen data. However, as a fortunate partition might happen and generalization error might be underestimated, instead of using a single validation set we use K validation sets and we use the mean error in this K sets as the CV-error. This CV-error should give us a reliable estimation of the generalization error.

But, how can we use it to obtain the optimal hyperparameters of the model? If the CV-error is an estimation of the generalization error, the hyperparameter values with least CV-error would result in the least generalization error. As CV-error can be computed during training, we obtain a faster way to obtain the optimal values of the hyperparameters.

Now that we know why we want cross validation, implement the `cross_validation` function. This function shall implement two cross validation methods:
- K-Fold cross validation
- Leave-one-out cross validation

Check this [link](https://machinelearningmastery.com/k-fold-cross-validation/) to know the details of each method.

In [8]:
from src.Lab2_6_CV import cross_validation

Now that we have the `cross_validation` function implemented, let's check which is the optimal value for `n_neighbors` in this problem.

In [ ]:
# Initialize lists to store mean scores and standard deviations for each value of k
mean_scores = []
std_scores = []

# Define the range of k values to test
k_values = range(4, 80)

# TODO: Loop through each value of k, obtaining the mean accuracy score and the standard deviation of the accuracy score in cross validation
for k in k_values:
    model = KNeighborsClassifier(n_neighbors=k)
    mean_loo, std_loo = cross_validation(model, X_train, y_train, -1)
    mean_k_fold, std_k_fold = cross_validation(model, X_train, y_train, 10)
    mean_scores.append([mean_loo, mean_k_fold])
    std_scores.append([std_loo, std_k_fold])
# TODO: Find the highest score and the corresponding optimal k
mean_scores = np.array(mean_scores)
mean_scores[0]


array([[0.76875, 0.765  ],
       [0.81375, 0.81375],
       [0.795  , 0.795  ],
       [0.81375, 0.81875],
       [0.7975 , 0.79625],
       [0.81625, 0.82   ],
       [0.79875, 0.79875],
       [0.8125 , 0.81625],
       [0.81   , 0.80375],
       [0.81375, 0.81125],
       [0.8075 , 0.80125],
       [0.81125, 0.80125],
       [0.79875, 0.79875],
       [0.8    , 0.80375],
       [0.8025 , 0.79875],
       [0.8025 , 0.8    ],
       [0.7975 , 0.79375],
       [0.80125, 0.79625],
       [0.79875, 0.79625],
       [0.8025 , 0.795  ],
       [0.7975 , 0.79   ],
       [0.79625, 0.78875],
       [0.78875, 0.7875 ],
       [0.79375, 0.795  ],
       [0.79   , 0.7875 ],
       [0.79125, 0.7875 ],
       [0.7875 , 0.7875 ],
       [0.79125, 0.78875],
       [0.79   , 0.785  ],
       [0.79375, 0.79   ],
       [0.78875, 0.785  ],
       [0.79   , 0.79125],
       [0.7875 , 0.78625],
       [0.79   , 0.78875],
       [0.78625, 0.78625],
       [0.7875 , 0.78875],
       [0.78375, 0.78625],
 

In [ ]:
list(mean_scores[:,-1:].reshape(-1))


[0.765,
 0.81375,
 0.7950000000000002,
 0.81875,
 0.79625,
 0.82,
 0.79875,
 0.8162499999999999,
 0.80375,
 0.8112499999999999,
 0.8012499999999999,
 0.8012500000000001,
 0.7987500000000001,
 0.80375,
 0.79875,
 0.8,
 0.79375,
 0.79625,
 0.7962499999999999,
 0.795,
 0.7899999999999999,
 0.7887500000000001,
 0.7875,
 0.7949999999999999,
 0.7875,
 0.7874999999999999,
 0.7875000000000001,
 0.7887500000000001,
 0.785,
 0.7899999999999999,
 0.785,
 0.79125,
 0.78625,
 0.78875,
 0.78625,
 0.78875,
 0.78625,
 0.7875,
 0.78625,
 0.7887500000000001,
 0.7825,
 0.7887500000000001,
 0.785,
 0.7875000000000001,
 0.7849999999999999,
 0.7887500000000001,
 0.78625,
 0.7887500000000001,
 0.7837500000000001,
 0.7837500000000001,
 0.77875,
 0.7837500000000001,
 0.7837500000000001,
 0.78,
 0.78,
 0.7775,
 0.78,
 0.77875,
 0.7775000000000001,
 0.7812500000000001,
 0.7775000000000001,
 0.78,
 0.7775000000000001,
 0.7787499999999999,
 0.7750000000000001,
 0.7775,
 0.7775000000000001,
 0.775,
 0.7775,
 0.775,

Now that we know the optimal value for `n_neighbors`, let's train a KNN model with this value of hyperparameter and check if the generalization error has improved.

In [ ]:
# TODO: Train model with k=optimal_k
model = KNeighborsClassifier(n_neighbors=optimal_k)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=9)

In [ ]:
# TODO: Calculate accuracy in training and test for KNN with k=optimal_k
acc_train = accuracy_score(model.predict(X_train), y_train)
acc_test = accuracy_score(model.predict(X_test), y_test)
print(f'Accuracy, train = {acc_train} test = {acc_test}')

Accuracy, train = 0.84125 test = 0.815


### Sensitivity analysis
- Does the number of folds affects the optimal value of `n_neighbors`? Why or why not?
> Sí, el valor óptimo del número de vecinos depende del número de folds, ya que cuanto mayor este último mejor será la estimación del error

- What happens with the computational time if you increment the number of folds?
> El coste computional aumenta al incrementar el número de folds, ya que cuanto mayor sea este último, en más subconjuntos tendrá que separar el conjunto de entrenamiento, y por tanto, más precisiones distintas deberá calcular

- Does it worth to increase the number of folds? Is the CV error a better proxy of the test set?
> Aunque la estimación del error de test mejora, el coste computacional aumenta demasiado, por lo que puede no ser conveniente aumentar el número de folds en exceso para cross validation